# **EOS: Novozymes Enzyme Stability Prediction**

In [ ]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import make_friedman2
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from google.colab import drive
from zipfile import ZipFile
import plotly.express as px
import re 

drive.mount('/content/drive')

TrainDf = pd.read_csv('/content/drive/MyDrive/RE_MACHINE_LEARNING_22/train.csv')
TestDf = pd.read_csv('/content/drive/MyDrive/RE_MACHINE_LEARNING_22/test (3).csv')





#feature Engineering
TrainDf.insert(2, 'sequence_length', len(TrainDf['protein_sequence']))
TrainDf.insert(4, 'IsNuetral', np.where(TrainDf['pH'] ==7.0, 1, 0))
TrainDf.insert(4, 'IsBasic', np.where(TrainDf['pH']>7.0, 1, 0))
TrainDf.insert(4, 'IsAcidic', np.where(TrainDf['pH']<7.0, 1, 0))
search_amino=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
for amino_acid in search_amino:
     TrainDf[amino_acid]=TrainDf['protein_sequence'].str.count(amino_acid,re.I)
TrainDf.drop(['protein_sequence'],axis=1,inplace=True)

TrainDf

TestDf.insert(2, 'sequence_length', len(TestDf['protein_sequence']))
TestDf.insert(4, 'IsNuetral', np.where(TestDf['pH'] ==7.0, 1, 0))
TestDf.insert(4, 'IsBasic', np.where(TestDf['pH']>7.0, 1, 0))
TestDf.insert(4, 'IsAcidic', np.where(TestDf['pH']<7.0, 1, 0))
search_amino=['A', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'K', 'L', 'M', 'N', 'P', 'Q', 'R', 'S', 'T', 'V', 'W', 'Y']
for amino_acid in search_amino:
     TestDf[amino_acid]=TestDf['protein_sequence'].str.count(amino_acid,re.I)
TestDf.drop(['protein_sequence'],axis=1,inplace=True)

TestDf


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,seq_id,sequence_length,pH,IsAcidic,IsBasic,IsNuetral,data_source,A,C,D,...,M,N,P,Q,R,S,T,V,W,Y
0,31390,2413,8,0,1,0,Novozymes,22,4,15,...,0,19,17,13,3,18,8,13,6,6
1,31391,2413,8,0,1,0,Novozymes,22,4,15,...,0,19,17,13,3,18,8,13,6,6
2,31392,2413,8,0,1,0,Novozymes,22,4,15,...,0,19,17,13,3,18,8,13,6,6
3,31393,2413,8,0,1,0,Novozymes,22,5,15,...,0,19,17,13,3,18,8,13,6,6
4,31394,2413,8,0,1,0,Novozymes,22,4,15,...,0,19,17,13,3,18,8,13,6,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2408,33798,2413,8,0,1,0,Novozymes,21,4,15,...,0,19,17,13,3,18,8,13,6,6
2409,33799,2413,8,0,1,0,Novozymes,21,4,15,...,0,19,17,13,3,18,8,13,6,6
2410,33800,2413,8,0,1,0,Novozymes,21,4,15,...,0,20,17,13,3,18,8,13,6,6
2411,33801,2413,8,0,1,0,Novozymes,21,4,15,...,0,19,18,13,3,18,8,13,6,6


# **EDA**

In [ ]:
TrainDf.dtypes

seq_id               int64
sequence_length      int64
pH                 float64
IsAcidic             int64
IsBasic              int64
IsNuetral            int64
data_source         object
tm                 float64
A                    int64
C                    int64
D                    int64
E                    int64
F                    int64
G                    int64
H                    int64
I                    int64
K                    int64
L                    int64
M                    int64
N                    int64
P                    int64
Q                    int64
R                    int64
S                    int64
T                    int64
V                    int64
W                    int64
Y                    int64
dtype: object

In [ ]:
TrainDf.describe()

,seq_id,sequence_length,pH,IsAcidic,IsBasic,IsNuetral,tm,A,C,D,...,M,N,P,Q,R,S,T,V,W,Y
count,31390.000000,31390.0,31104.000000,31390.000000,31390.000000,31390.000000,31390.000000,31390.000000,31390.000000,31390.000000,...,31390.000000,31390.000000,31390.000000,31390.000000,31390.000000,31390.000000,31390.000000,31390.000000,31390.000000,31390.000000
mean,15694.500000,31390.0,6.892339,0.083753,0.064129,0.843007,49.147337,34.297579,6.605607,24.880280,...,10.306021,18.847021,22.532431,19.437623,23.757662,32.176521,24.176649,29.247372,5.025613,13.266518
std,9061.656811,0.0,1.612225,0.277021,0.244986,0.363800,14.010089,43.774015,12.763594,35.269664,...,11.179000,25.158768,44.987207,28.756956,33.269851,50.387319,43.938906,53.060187,8.888754,18.562705
min,0.000000,31390.0,1.990000,0.000000,0.000000,0.000000,-1.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7847.250000,31390.0,7.000000,0.000000,0.000000,1.000000,42.100000,15.000000,2.000000,10.000000,...,4.000000,8.000000,7.000000,7.000000,9.000000,11.000000,10.000000,13.000000,1.000000,6.000000
50%,15694.500000,31390.0,7.000000,0.000000,0.000000,1.000000,48.000000,26.000000,4.000000,18.000000,...,8.000000,13.000000,15.000000,12.000000,17.000000,21.000000,17.000000,22.000000,3.000000,10.000000
75%,23541.750000,31390.0,7.000000,0.000000,0.000000,1.000000,53.800000,43.000000,8.000000,30.000000,...,13.000000,22.000000,26.000000,22.000000,29.000000,37.000000,29.000000,35.000000,6.000000,17.000000
max,31389.000000,31390.0,64.900000,1.000000,1.000000,1.000000,130.000000,1989.000000,498.000000,1660.000000,...,367.000000,1081.000000,2439.000000,870.000000,1532.000000,2268.000000,2416.000000,3091.000000,456.000000,948.000000


In [ ]:
TrainDf.isna().sum()
TestDf.isna().sum()

seq_id             0
sequence_length    0
pH                 0
IsAcidic           0
IsBasic            0
IsNuetral          0
data_source        0
A                  0
C                  0
D                  0
E                  0
F                  0
G                  0
H                  0
I                  0
K                  0
L                  0
M                  0
N                  0
P                  0
Q                  0
R                  0
S                  0
T                  0
V                  0
W                  0
Y                  0
dtype: int64

In [ ]:
TrainDf['pH'].fillna(TrainDf['pH'].mean(), inplace=True)
TrainDf['data_source'].fillna(TrainDf['data_source'][0], inplace=True)



In [ ]:
TrainDf.isna().sum()


seq_id             0
sequence_length    0
pH                 0
IsAcidic           0
IsBasic            0
IsNuetral          0
data_source        0
tm                 0
A                  0
C                  0
D                  0
E                  0
F                  0
G                  0
H                  0
I                  0
K                  0
L                  0
M                  0
N                  0
P                  0
Q                  0
R                  0
S                  0
T                  0
V                  0
W                  0
Y                  0
dtype: int64

# **Modeling**

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from scipy.sparse import hstack


Y= TrainDf['tm']
TrainDf.drop(['tm'],axis=1,inplace=True)
TrainDf.drop(['data_source'],axis=1,inplace=True)
TestDf.drop(['data_source'],axis=1,inplace=True)
X = TrainDf
display(X)
print(X.shape)
print(TestDf.shape)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.15)
print(X.shape)
print(TestDf.shape)

,seq_id,sequence_length,pH,IsAcidic,IsBasic,IsNuetral,A,C,D,E,...,M,N,P,Q,R,S,T,V,W,Y
0,0,31390,7.0,0,0,1,45,1,13,30,...,8,5,18,6,25,11,14,37,4,3
1,1,31390,7.0,0,0,1,28,0,10,52,...,2,6,8,22,30,14,12,13,3,3
2,2,31390,7.0,0,0,1,50,9,27,32,...,6,15,20,25,31,33,30,30,3,16
3,3,31390,7.0,0,0,1,20,5,19,29,...,2,9,16,9,10,16,19,14,3,4
4,4,31390,7.0,0,0,1,86,14,78,78,...,31,65,128,54,63,148,120,124,16,47
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31385,31385,31390,7.0,0,0,1,33,12,38,31,...,13,24,25,24,42,33,18,42,13,18
31386,31386,31390,7.0,0,0,1,37,5,21,29,...,14,19,19,16,25,37,26,34,5,14
31387,31387,31390,7.0,0,0,1,13,1,7,7,...,7,5,6,8,3,10,6,7,4,4
31388,31388,31390,7.0,0,0,1,47,5,34,36,...,26,25,31,12,25,51,32,48,3,18


(31390, 26)
(2413, 26)
(31390, 26)
(2413, 26)


Method to Evaluate Models

In [ ]:
def evaluateModel(Xtrain, Ytrain, Xtest, Ytest, model):
  model.fit(Xtrain, Ytrain)
  score = model.score(Xtrain, Ytrain) 
  print("Model Training score: %.2f" %score)
  #kfold = KFold(n_splits=10, shuffle=True)
  #kf_cv_scores = cross_val_score(model, Xtrain, Ytrain, cv=kfold )
  #print("K-fold CV average score: %.2f" % kf_cv_scores.mean())
  ypred = model.predict(Xtest)
  mse = mean_squared_error(Ytest, ypred)
  print("MSE: %.2f" % mse)
  smape = 1/len(Ytest) * np.sum(2*np.abs(ypred-Ytest)/(np.abs(Ytest)+np.abs(ypred))*100)
  print("SMAPE: %.2f" % smape)
  x_simple = pd.DataFrame({'Actual':Ytest,
                           'Prediction': ypred
                           })
  display(x_simple.corr(method="spearman"))

Testing XGB Regressor

In [ ]:
from xgboost import XGBRegressor
xgbr = XGBRegressor(
    objective = 'reg:squarederror',
    colsample_bytree = 0.5,
    learning_rate = 0.05,
    max_depth = 6,
    min_child_weight = 1,
    n_estimators = 1000,
    subsample = 0.7
)

evaluateModel(X_train, Y_train, X_test, Y_test, xgbr)
pipeline = make_pipeline(xgbr)
pipeline.fit(X,Y)
answers = pipeline.predict(TestDf)
submisssionFile = pd.DataFrame({'seq_id': TestDf['seq_id'], 'tm': answers}).to_csv('FileSubmission.csv', index=False)


Model Training score: 0.81
MSE: 77.28
SMAPE: 13.16


,Actual,Prediction
Actual,1.000000,0.584199
Prediction,0.584199,1.000000


Decision Tree Regressor

In [ ]:
DecTreeReg = DecisionTreeRegressor(random_state=42)
evaluateModel(X_train, Y_train, X_test, Y_test, DecTreeReg)
pipeline = make_pipeline(DecTreeReg)
pipeline.fit(X,Y)
answers = pipeline.predict(TestDf)
submisssionFile = pd.DataFrame({'seq_id': TestDf['seq_id'], 'tm': answers}).to_csv('FileSubmission.csv', index=False)

Model Training score: 1.00
MSE: 171.01
SMAPE: 17.42


,Actual,Prediction
Actual,1.000000,0.405222
Prediction,0.405222,1.000000


Random Forest Regressor

In [ ]:
regressor = RandomForestRegressor(max_depth = 700,
              n_estimators=10000,
              max_features = 12,
              min_samples_leaf= 5,
              min_samples_split = 7,
              random_state = 42)
evaluateModel(X_train, Y_train, X_test, Y_test, regressor)
pipeline = make_pipeline(regressor)
pipeline.fit(X,Y)
answers = pipeline.predict(TestDf)
submisssionFile = pd.DataFrame({'seq_id': TestDf['seq_id'], 'tm': answers}).to_csv('FileSubmission.csv', index=False)

Model Training score: 0.79
MSE: 80.41
SMAPE: 13.49


,Actual,Prediction
Actual,1.000000,0.591871
Prediction,0.591871,1.000000


In [ ]:
from scipy.stats import randint
from sklearn.model_selection import RandomizedSearchCV
random_grid={'max_depth':list(np.arange(100, 1000, step=100)),
              'n_estimators':np.arange(100, 10000, step=100),
              'max_features':randint(1,15),
              'min_samples_leaf':randint(1,7),
              'min_samples_split':np.arange(2, 10, step=1),
              'min_impurity_decrease':np.arange(2, 10, step=1)
         }
regressor = RandomForestRegressor(random_state=42)
# Random search of parameters, using 3 fold cross validation, 
rf_random = RandomizedSearchCV(estimator=regressor, param_distributions=random_grid,
                              scoring='neg_mean_absolute_error', 
                              cv =5, verbose=2, random_state=42,
                              return_train_score=True)

# Fit the random search model
rf_random.fit(X_train,Y_train)
print("Best parameters:", rf_random.best_params_)
print("Lowest MAE: ", (-rf_random.best_score_))


In [ ]:
regressor = RandomForestRegressor(param_grid = rf_random.best_params_)
regressor.fit(X, Y)
ypred = regressor.predict(X_test)
x_simple = pd.DataFrame({'Actual':Y_test,
                           'Prediction': ypred
                           })
display(x_simple.corr(method="spearman"))
pipeline = make_pipeline(xgbr)
pipeline.fit(X,Y)
answers = pipeline.predict(TestDf)
submisssionFile = pd.DataFrame({'seq_id': TestDf['seq_id'], 'tm': answers}).to_csv('FileSubmission.csv', index=False)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import log_loss, make_scorer

params = { 'max_depth': [2,10],
            'nthreads':[10, 100, 1000],
           'learning_rate': [0.05, 0.1],
           'n_estimators': [100, 1000],
           'colsample_bytree': [0.01, 0.5],
           'min_child_weight':[5,10],
           'gamma':[0.1,  1],
           'objective':['reg:squarederror']

          }
xgbr = xgb.XGBRegressor(seed = 42)
clf = GridSearchCV(estimator=xgbr, 
                   param_grid=params, 
                   verbose=3, cv=2)
clf.fit(X, Y)
print(clf.best_params_)


In [ ]:
xgbr = xgb.XGBRegressor(param_grid = clf.best_params_, seed = 42)
xgbr.fit(X, Y)
ypred = xgbr.predict(X_test)
x_simple = pd.DataFrame({'Actual':Y_test,
                           'Prediction': ypred
                           })
display(x_simple.corr(method="spearman"))
pipeline = make_pipeline(xgbr)
pipeline.fit(X,Y)
answers = pipeline.predict(TestDf)
submisssionFile = pd.DataFrame({'seq_id': TestDf['seq_id'], 'tm': answers}).to_csv('FileSubmission.csv', index=False)
